<cell_type>markdown</cell_type># TerraClimate - Extraction des Données Climatiques

## C'est quoi TerraClimate ?

**TerraClimate** est une base de données climatiques mondiale qui contient :
- Des données **mensuelles** depuis 1958
- Une résolution spatiale de **~4 km**
- Des variables comme : température, précipitations, évapotranspiration, etc.

## Pourquoi on en a besoin ?

La qualité de l'eau dépend du **climat** :
- **Évaporation** → concentre les minéraux → affecte l'alcalinité et la conductivité
- **Précipitations** → ruissellement des engrais → affecte le phosphore
- **Température** → croissance des algues → indique la pollution

## Ce que fait ce notebook

```
1. Se connecter à l'API Microsoft Planetary Computer
         ↓
2. Télécharger les données TerraClimate (2011-2015, Afrique du Sud)
         ↓
3. Pour chaque site de mesure d'eau :
   - Trouver le point de grille TerraClimate le plus proche
   - Extraire la valeur climatique à la date correspondante
         ↓
4. Sauvegarder le fichier CSV avec les features climatiques
```

## Source des données

Documentation : [TerraClimate sur Planetary Computer](https://planetarycomputer.microsoft.com/dataset/terraclimate#overview)

<cell_type>markdown</cell_type>---

## Étape 1 : Installation des dépendances

**Première exécution uniquement** : Après avoir exécuté cette cellule, il faut redémarrer le kernel :
1. Cliquer sur "Connected" en haut
2. Sélectionner "Restart kernel"

Les exécutions suivantes n'ont pas besoin de ce redémarrage.

In [2]:
!pip install uv
!uv pip install --system -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


^C


In [12]:
# =============================================================================
# IMPORTS
# =============================================================================

# Snowflake (uniquement si le notebook tourne dans Snowflake Notebooks)
# Commenter ces lignes pour exécuter en local
# import snowflake
# from snowflake.snowpark.context import get_active_session
# session = get_active_session()

import warnings
warnings.filterwarnings("ignore")

# Manipulation de données
import numpy as np
import pandas as pd

# xarray : pour manipuler des données multidimensionnelles (climat, satellite)
# C'est comme pandas, mais pour des données avec plusieurs dimensions (lat, lon, temps)
import xarray as xr

# Pour trouver le point le plus proche (algorithme KD-Tree)
from scipy.spatial import cKDTree

# Accès à l'API Microsoft Planetary Computer
import pystac_client      # Pour naviguer dans le catalogue de données
import planetary_computer as pc  # Pour s'authentifier

from datetime import date
from tqdm import tqdm  # Barre de progression
import os

print("Imports OK!")

Imports OK!


<cell_type>markdown</cell_type>---

## Étape 2 : Définition des fonctions

On va créer 3 fonctions :

| Fonction | Rôle |
|----------|------|
| `load_terraclimate_dataset()` | Se connecte à l'API et charge le dataset complet |
| `filterg(ds, var)` | Filtre par zone géographique et période |
| `assign_nearest_climate()` | Associe chaque site de mesure au point climatique le plus proche |

<cell_type>markdown</cell_type>### Fonction 1 : Charger le dataset TerraClimate

Cette fonction se connecte à **Microsoft Planetary Computer** et ouvre le dataset TerraClimate.

**Format Zarr** : C'est un format optimisé pour les gros datasets climatiques. 
- Permet de ne charger que les parties dont on a besoin (lazy loading)
- Plus rapide que NetCDF pour les accès partiels

In [13]:
def load_terraclimate_dataset():
    """
    Charge le dataset TerraClimate depuis Microsoft Planetary Computer.
    
    Retourne :
        xarray.Dataset contenant toutes les variables climatiques
    
    Variables disponibles dans TerraClimate :
        - pet : Potential Evapotranspiration (évapotranspiration potentielle)
        - ppt : Precipitation (précipitations)
        - tmax : Maximum Temperature (température max)
        - tmin : Minimum Temperature (température min)
        - soil : Soil Moisture (humidité du sol)
        - et d'autres...
    """
    
    # 1. Se connecter au catalogue Planetary Computer
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=pc.sign_inplace,  # Authentification automatique
    )
    
    # 2. Récupérer la collection TerraClimate
    collection = catalog.get_collection("terraclimate")
    asset = collection.assets["zarr-abfs"]  # Format Zarr sur Azure Blob Storage

    # 3. Ouvrir le dataset (avec les bonnes options de stockage)
    if "xarray:storage_options" in asset.extra_fields:
        ds = xr.open_zarr(
            asset.href,
            storage_options=asset.extra_fields["xarray:storage_options"],
            consolidated=True,
        )
    else:
        ds = xr.open_dataset(
            asset.href,
            **asset.extra_fields["xarray:open_kwargs"],
        )

    print(f"Dataset chargé ! Variables disponibles : {list(ds.data_vars)}")
    return ds

In [ ]:
<cell_type>markdown</cell_type>### Fonction 2 : Filtrer par zone et période

Le dataset TerraClimate couvre le **monde entier** depuis 1958. On ne veut que :
- **Période** : 2011-2015 (nos données de qualité d'eau)
- **Zone** : Afrique du Sud (lat: -35 à -22, lon: 15 à 33)

```
    Monde entier                    Afrique du Sud seulement
    ┌─────────────────┐             ┌─────────────────┐
    │ □ □ □ □ □ □ □ □ │             │                 │
    │ □ □ □ □ □ □ □ □ │   filtrer   │     ┌─────┐     │
    │ □ □ □ □ □ □ □ □ │  ────────►  │     │ □ □ │     │
    │ □ □ □ □ □ □ □ □ │             │     └─────┘     │
    └─────────────────┘             └─────────────────┘
```

In [14]:
def filterg(ds, var):
    """
    Filtre le dataset TerraClimate pour une variable, une période et une zone.
    VERSION OPTIMISÉE : utilise le slicing xarray au lieu d'une boucle.

    Paramètres :
        ds : xarray.Dataset - le dataset TerraClimate complet
        var : str - nom de la variable à extraire ('pet', 'ppt', 'tmax', etc.)

    Retourne :
        DataFrame avec colonnes : Latitude, Longitude, Sample Date, [variable]
    """

    # Filtrer directement avec xarray (BEAUCOUP plus rapide qu'une boucle)
    # - Période : 2011-2015
    # - Zone : Afrique du Sud (lat: -35.18 à -21.72, lon: 14.97 à 32.79)
    # Note: les latitudes vont du nord au sud, donc on inverse l'ordre du slice
    ds_filtered = ds[var].sel(
        time=slice("2011-01-01", "2015-12-31"),
        lat=slice(-21.72, -35.18),  # Nord vers Sud (ordre décroissant)
        lon=slice(14.97, 32.79)
    )

    print(f"Période : 2011-2015 ({len(ds_filtered.time)} mois)")
    print(f"Zone : lat [{ds_filtered.lat.min().values:.2f}, {ds_filtered.lat.max().values:.2f}], lon [{ds_filtered.lon.min().values:.2f}, {ds_filtered.lon.max().values:.2f}]")

    # Convertir en DataFrame en une seule opération
    print(f"Conversion en DataFrame pour {var}...")
    df_final = ds_filtered.to_dataframe().reset_index()

    # Convertir la date en string (pour compatibilité)
    df_final['time'] = df_final['time'].astype(str)

    # Renommer les colonnes pour correspondre à notre format
    df_final = df_final.rename(columns={
        "lat": "Latitude",
        "lon": "Longitude",
        "time": "Sample Date"
    })

    print(f"Extraction terminée : {len(df_final)} lignes")
    return df_final

In [ ]:
<cell_type>markdown</cell_type>### Fonction 3 : Associer chaque site au point climatique le plus proche

**Problème** : TerraClimate a une grille de ~4km. Nos sites de mesure ne tombent pas exactement sur cette grille.

**Solution** : Pour chaque site, on trouve le point TerraClimate le plus proche avec un **KD-Tree**.

```
    Nos sites de mesure        Grille TerraClimate (~4km)
           •                        □───□───□───□
           •                        │   │   │   │
           •                        □───□───□───□
                                    │   │   │   │
                                    □───□───□───□
                    
                         ↓ KD-Tree
                         
    Site 1 → Point (lat=-28.5, lon=26.0) → pet = 175.2
    Site 2 → Point (lat=-29.0, lon=27.5) → pet = 182.1
    ...
```

**KD-Tree** : Algorithme efficace pour trouver le voisin le plus proche (complexité O(log n) au lieu de O(n)).

In [15]:
def assign_nearest_climate(sa_df, climate_df, var_name):
    """
    Associe chaque site de mesure d'eau à la valeur climatique la plus proche.
    VERSION OPTIMISÉE : utilise des opérations vectorisées au lieu de boucles.

    Paramètres :
        sa_df : DataFrame avec nos sites (Latitude, Longitude, Sample Date)
        climate_df : DataFrame avec les données TerraClimate
        var_name : str - nom de la variable climatique ('pet', 'ppt', etc.)

    Retourne :
        DataFrame avec une colonne contenant les valeurs climatiques
    """

    # ==========================================================================
    # ÉTAPE 1 : Construire le KD-Tree pour trouver les voisins rapidement
    # ==========================================================================

    # Copier pour éviter de modifier les originaux
    sa_df = sa_df.copy().reset_index(drop=True)
    climate_df = climate_df.copy()

    # Obtenir les coordonnées uniques du climat pour le KD-Tree
    climate_unique_coords = climate_df[['Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)

    # Convertir les coordonnées en radians (pour calcul de distance sur une sphère)
    sa_coords = np.radians(sa_df[['Latitude', 'Longitude']].values)
    climate_coords_unique = np.radians(climate_unique_coords.values)

    # Construire l'arbre KD avec les points climatiques uniques
    tree = cKDTree(climate_coords_unique)

    # Trouver le point le plus proche pour chaque site
    distances, indices = tree.query(sa_coords, k=1)

    # Récupérer les coordonnées des points les plus proches
    sa_df['nearest_lat'] = climate_unique_coords.iloc[indices]['Latitude'].values
    sa_df['nearest_lon'] = climate_unique_coords.iloc[indices]['Longitude'].values

    # ==========================================================================
    # ÉTAPE 2 : Associer les valeurs climatiques par mois (VECTORISÉ)
    # ==========================================================================

    # Convertir les dates et extraire le mois (YYYY-MM)
    sa_df['Sample Date'] = pd.to_datetime(sa_df['Sample Date'], dayfirst=True, errors='coerce')
    sa_df['year_month'] = sa_df['Sample Date'].dt.to_period('M')

    climate_df['Sample Date'] = pd.to_datetime(climate_df['Sample Date'], errors='coerce')
    climate_df['year_month'] = climate_df['Sample Date'].dt.to_period('M')

    # Merge sur (coordonnées les plus proches + mois)
    result = sa_df.merge(
        climate_df[['Latitude', 'Longitude', 'year_month', var_name]],
        left_on=['nearest_lat', 'nearest_lon', 'year_month'],
        right_on=['Latitude', 'Longitude', 'year_month'],
        how='left'
    )

    # Créer le DataFrame de sortie
    output_df = pd.DataFrame({var_name: result[var_name].values})

    print(f"Association terminée : {len(output_df)} valeurs, {output_df[var_name].isna().sum()} NaN")
    return output_df

---

## Étape 3 : Extraction pour les données d'entraînement

Maintenant on applique les fonctions pour extraire **toutes les variables climatiques** pertinentes pour chaque site de mesure.

### Variables TerraClimate extraites

| Variable | Description | Lien avec la qualité de l'eau |
|----------|-------------|-------------------------------|
| **pet** | Évapotranspiration potentielle | Climat sec → évaporation → concentration des minéraux |
| **aet** | Évapotranspiration réelle | Quantité d'eau réellement évaporée |
| **ppt** | Précipitations | Ruissellement → lessivage des sols → apport de nutriments |
| **tmax** | Température maximale | Croissance des algues, activité biologique |
| **tmin** | Température minimale | Stratification thermique, oxygénation |
| **soil** | Humidité du sol | Infiltration, transport de contaminants |
| **def** | Déficit hydrique | Stress hydrique → concentration des polluants |
| **pdsi** | Indice de sécheresse Palmer | Conditions de sécheresse prolongée |
| **vpd** | Déficit de pression de vapeur | Évaporation, stress végétal |
| **ws** | Vitesse du vent | Brassage de l'eau, évaporation |

> **Note** : La variable 'ro' (runoff) n'est pas disponible dans le dataset TerraClimate sur Planetary Computer.

In [16]:
# Charger les données de qualité d'eau (nos sites de mesure)
Water_Quality_df = pd.read_csv("water_quality_training_dataset.csv")

print(f"Nombre de sites : {len(Water_Quality_df)}")
print(f"Colonnes : {list(Water_Quality_df.columns)}")
display(Water_Quality_df.head())

Nombre de sites : 9319
Colonnes : ['Latitude', 'Longitude', 'Sample Date', 'Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus']


,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-28.760833,17.730278,02-01-2011,128.912,555.0,10.0
1,-26.861111,28.884722,03-01-2011,74.720,162.9,163.0
2,-26.450000,28.085833,03-01-2011,89.254,573.0,80.0
3,-27.671111,27.236944,03-01-2011,82.000,203.6,101.0
4,-27.356667,27.286389,03-01-2011,56.100,145.1,151.0


In [ ]:
# =============================================================================
# EXTRACTION DES DONNÉES TERRACLIMATE - TOUTES LES VARIABLES
# =============================================================================

# Définir le dossier de sortie (même dossier que le notebook)
import os
OUTPUT_DIR = r"c:\Documents\DATA SCIENCE\PROJET EY\Snowflake Notebooks Package"

# Liste des variables à extraire
# Note: 'ro' (runoff) n'est pas disponible dans le dataset TerraClimate sur Planetary Computer
TERRACLIMATE_VARIABLES = [
    'pet',   # Potential Evapotranspiration (évapotranspiration potentielle)
    'aet',   # Actual Evapotranspiration (évapotranspiration réelle)
    'ppt',   # Precipitation (précipitations)
    'tmax',  # Maximum Temperature (température max)
    'tmin',  # Minimum Temperature (température min)
    'soil',  # Soil Moisture (humidité du sol)
    'def',   # Climate Water Deficit (déficit hydrique)
    'pdsi',  # Palmer Drought Severity Index (indice de sécheresse)
    'vpd',   # Vapor Pressure Deficit (déficit de pression de vapeur)
    'ws',    # Wind Speed (vitesse du vent)
]

# 1. Charger le dataset TerraClimate complet
print("1. Connexion à Microsoft Planetary Computer...")
ds = load_terraclimate_dataset()

# 2. Extraire et associer chaque variable
print(f"\n2. Extraction de {len(TERRACLIMATE_VARIABLES)} variables climatiques...")

# Initialiser le DataFrame final avec les coordonnées
Terraclimate_training_df = Water_Quality_df[['Latitude', 'Longitude', 'Sample Date']].copy()

# Dictionnaire pour stocker les données climatiques filtrées (pour réutilisation avec validation)
tc_data_cache = {}

for var in TERRACLIMATE_VARIABLES:
    print(f"\n{'='*60}")
    print(f"Variable : {var.upper()}")
    print('='*60)
    
    # Filtrer les données pour cette variable
    tc_filtered = filterg(ds, var)
    tc_data_cache[var] = tc_filtered  # Stocker pour réutilisation
    
    # Associer aux sites de mesure
    var_df = assign_nearest_climate(Water_Quality_df, tc_filtered, var)
    
    # Ajouter au DataFrame final
    Terraclimate_training_df[var] = var_df[var].values

print("\n" + "="*60)
print("EXTRACTION TERMINÉE !")
print("="*60)

In [ ]:
# =============================================================================
# CRÉER LE FICHIER CSV FINAL
# =============================================================================

# Réorganiser les colonnes : coordonnées d'abord, puis variables climatiques
columns_order = ['Latitude', 'Longitude', 'Sample Date'] + TERRACLIMATE_VARIABLES
Terraclimate_training_df = Terraclimate_training_df[columns_order]

# Sauvegarder en CSV (chemin absolu pour être sûr)
output_path = os.path.join(OUTPUT_DIR, 'terraclimate_features_training.csv')
Terraclimate_training_df.to_csv(output_path, index=False)

print(f"Fichier créé : {output_path}")

In [17]:
# Aperçu du fichier créé
print("Aperçu des données extraites :")
print(f"- Lignes : {len(Terraclimate_training_df)}")
print(f"- Colonnes : {list(Terraclimate_training_df.columns)}")
print(f"\nStatistiques des variables climatiques :")
print(Terraclimate_training_df[TERRACLIMATE_VARIABLES].describe().round(2))
display(Terraclimate_training_df.head())

Aperçu des données extraites :
- Lignes : 9319
- Colonnes : ['Latitude', 'Longitude', 'Sample Date', 'pet', 'aet', 'ppt', 'tmax', 'tmin', 'soil', 'def', 'pdsi', 'vpd', 'ws']

Statistiques des variables climatiques :
           pet      aet      ppt     tmax     tmin     soil      def     pdsi  \
count  9319.00  9319.00  9319.00  9319.00  9319.00  9319.00  9319.00  9319.00   
mean    124.75    39.47    41.89    26.07    10.38     7.96    85.28    -1.24   
std      42.67    37.71    44.91     4.39     5.60    13.66    50.52     2.47   
min      42.10     0.00     0.00    12.22    -3.61     0.00     0.00    -6.08   
25%      86.90     8.30     7.30    23.00     6.24     1.00    52.70    -2.92   
50%     122.90    29.40    28.40    26.49    11.12     3.10    82.70    -2.02   
75%     155.90    58.10    59.90    29.29    14.79     8.50   117.70    -0.29   
max     270.80   173.40   374.50    38.12    22.98   162.20   267.20     8.19   

           vpd       ws  
count  9319.00  9319.00  
me

,Latitude,Longitude,Sample Date,pet,aet,ppt,tmax,tmin,soil,def,pdsi,vpd,ws
0,-28.760833,17.730278,02-01-2011,236.300003,7.900000,8.3,35.759998,21.029999,0.000000,228.400009,-1.16,2.82,2.94
1,-26.861111,28.884722,03-01-2011,134.300003,134.300003,175.0,25.410000,14.099999,35.700001,0.000000,1.54,0.80,1.93
2,-26.450000,28.085833,03-01-2011,141.500000,141.500000,156.1,26.070000,15.250000,9.900001,0.000000,-0.62,0.93,2.15
3,-27.671111,27.236944,03-01-2011,152.400009,152.400009,194.7,27.599998,15.960000,28.200001,0.000000,3.08,1.18,2.22
4,-27.356667,27.286389,03-01-2011,151.900009,151.900009,180.4,27.209999,16.150000,21.300001,0.000000,2.69,1.12,2.27


In [ ]:
# Sauvegarder dans le dossier temporaire (uniquement pour Snowflake)
# Commenter cette ligne pour exécuter en local
# Terraclimate_training_df.to_csv("/tmp/terraclimate_features_training.csv", index=False)

In [ ]:
# Uploader le fichier vers Snowflake (uniquement si le notebook tourne dans Snowflake)
# Commenter ces lignes pour exécuter en local
# session.sql(f"""
#     PUT file:///tmp/terraclimate_features_training.csv
#     snow://workspace/USER$.PUBLIC.DEFAULT$/versions/live/
#     AUTO_COMPRESS=FALSE
#     OVERWRITE=TRUE
# """).collect()

# print("Fichier uploadé vers Snowflake !")
# print("Rafraîchissez le navigateur pour voir le fichier dans la barre latérale.")

print("Fichier 'terraclimate_features_training.csv' sauvegardé localement !")

<cell_type>markdown</cell_type>---

## Étape 4 : Extraction pour les données de validation (test)

Même processus, mais pour les données de **validation** (le fichier `submission_template.csv`).

C'est ce fichier qu'on utilisera pour faire nos prédictions finales.

In [18]:
# Charger le template de soumission (sites de validation)
Validation_df = pd.read_csv('submission_template.csv')

print(f"Nombre de sites de validation : {len(Validation_df)}")
display(Validation_df.head())

Nombre de sites de validation : 200


,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-32.043333,27.822778,01-09-2014,NaN,NaN,NaN
1,-33.329167,26.077500,16-09-2015,NaN,NaN,NaN
2,-32.991639,27.640028,07-05-2015,NaN,NaN,NaN
3,-34.096389,24.439167,07-02-2012,NaN,NaN,NaN
4,-32.000556,28.581667,01-10-2014,NaN,NaN,NaN


In [20]:
# =============================================================================
# EXTRACTION POUR LES DONNÉES DE VALIDATION - TOUTES LES VARIABLES
# =============================================================================

# Initialiser le DataFrame final avec les coordonnées
Terraclimate_validation_df = Validation_df[['Latitude', 'Longitude', 'Sample Date']].copy()

print(f"Extraction de {len(TERRACLIMATE_VARIABLES)} variables pour la validation...")

for var in TERRACLIMATE_VARIABLES:
    print(f"\n{'='*60}")
    print(f"Variable : {var.upper()}")
    print('='*60)
    
    # Réutiliser les données climatiques filtrées (depuis le cache)
    tc_filtered = tc_data_cache[var]
    
    # Associer aux sites de validation
    var_df = assign_nearest_climate(Validation_df, tc_filtered, var)
    
    # Ajouter au DataFrame final
    Terraclimate_validation_df[var] = var_df[var].values

print("\n" + "="*60)
print("EXTRACTION VALIDATION TERMINÉE !")
print("="*60)

Extraction de 10 variables pour la validation...

Variable : PET
Association terminée : 200 valeurs, 0 NaN

Variable : AET
Association terminée : 200 valeurs, 0 NaN

Variable : PPT
Association terminée : 200 valeurs, 0 NaN

Variable : TMAX
Association terminée : 200 valeurs, 0 NaN

Variable : TMIN
Association terminée : 200 valeurs, 0 NaN

Variable : SOIL
Association terminée : 200 valeurs, 0 NaN

Variable : DEF
Association terminée : 200 valeurs, 0 NaN

Variable : PDSI
Association terminée : 200 valeurs, 0 NaN

Variable : VPD
Association terminée : 200 valeurs, 0 NaN

Variable : WS
Association terminée : 200 valeurs, 0 NaN

EXTRACTION VALIDATION TERMINÉE !


In [22]:
# Créer le fichier CSV de validation
# Réorganiser les colonnes : coordonnées d'abord, puis variables climatiques
columns_order = ['Latitude', 'Longitude', 'Sample Date'] + TERRACLIMATE_VARIABLES
Terraclimate_validation_df = Terraclimate_validation_df[columns_order]

# Sauvegarder (chemin absolu)
output_path = os.path.join(OUTPUT_DIR, 'terraclimate_features_validation.csv')
Terraclimate_validation_df.to_csv(output_path, index=False)

print(f"Fichier créé : {output_path}")

Fichier créé : c:\Documents\DATA SCIENCE\PROJET EY\Snowflake Notebooks Package\terraclimate_features_validation.csv


In [23]:
# Aperçu des données de validation
print(f"Données de validation : {len(Terraclimate_validation_df)} lignes")
print(f"Colonnes : {list(Terraclimate_validation_df.columns)}")
print(f"\nStatistiques des variables climatiques :")
print(Terraclimate_validation_df[TERRACLIMATE_VARIABLES].describe().round(2))
display(Terraclimate_validation_df.head())

Données de validation : 200 lignes
Colonnes : ['Latitude', 'Longitude', 'Sample Date', 'pet', 'aet', 'ppt', 'tmax', 'tmin', 'soil', 'def', 'pdsi', 'vpd', 'ws']

Statistiques des variables climatiques :
          pet     aet     ppt    tmax    tmin    soil     def    pdsi     vpd  \
count  200.00  200.00  200.00  200.00  200.00  200.00  200.00  200.00  200.00   
mean   113.08   47.84   50.09   24.42   11.70   12.21   65.24   -0.27    0.91   
std     33.45   31.70   40.37    3.35    3.96   14.35   44.97    1.68    0.26   
min     55.70    1.10    1.20   15.48   -0.85    0.10    0.00   -4.20    0.37   
25%     86.02   21.80   18.85   22.29    9.30    0.40   28.78   -1.45    0.76   
50%    108.35   43.80   38.55   24.34   12.00    8.50   64.40   -0.87    0.86   
75%    133.55   64.33   66.40   26.43   14.48   16.47   87.00    1.18    1.00   
max    214.20  128.20  183.00   33.98   18.90   74.50  207.40    4.72    1.84   

           ws  
count  200.00  
mean     3.36  
std      0.74  
min 

,Latitude,Longitude,Sample Date,pet,aet,ppt,tmax,tmin,soil,def,pdsi,vpd,ws
0,-32.043333,27.822778,01-09-2014,114.900002,30.400000,31.0,24.340000,9.66,7.1,84.500000,-1.49,1.08,2.74
1,-33.329167,26.077500,16-09-2015,104.900002,34.900002,36.5,24.000000,8.91,3.2,70.000000,-0.07,0.90,3.15
2,-32.991639,27.640028,07-05-2015,84.400002,30.600000,22.1,24.369999,12.70,24.5,53.799999,-2.15,0.90,3.39
3,-34.096389,24.439167,07-02-2012,112.900002,82.900002,85.9,24.330000,16.34,8.3,30.000000,2.43,0.48,4.35
4,-32.000556,28.581667,01-10-2014,131.000000,18.100000,18.0,25.029999,12.79,6.4,112.900002,-2.45,0.92,3.81


In [ ]:
# Sauvegarder dans /tmp pour Snowflake (uniquement si le notebook tourne dans Snowflake)
# Commenter cette ligne pour exécuter en local
# Terraclimate_validation_df.to_csv("/tmp/terraclimate_features_validation.csv", index=False)

In [ ]:
# Uploader vers Snowflake (uniquement si le notebook tourne dans Snowflake)
# Commenter ces lignes pour exécuter en local
# session.sql(f"""
#     PUT file:///tmp/terraclimate_features_validation.csv
#     snow://workspace/USER$.PUBLIC.DEFAULT$/versions/live/
#     AUTO_COMPRESS=FALSE
#     OVERWRITE=TRUE
# """).collect()

# print("Fichier uploadé vers Snowflake !")

print("Fichier 'terraclimate_features_validation.csv' sauvegardé localement !")

---

## Résumé

**Ce qu'on a fait :**
1. Connecté à Microsoft Planetary Computer
2. Téléchargé les données TerraClimate pour l'Afrique du Sud (2011-2015)
3. Extrait **10 variables climatiques** pour chaque site de mesure
4. Créé 2 fichiers CSV avec toutes les features

**Variables extraites :**

| Variable | Description |
|----------|-------------|
| pet | Évapotranspiration potentielle |
| aet | Évapotranspiration réelle |
| ppt | Précipitations |
| tmax | Température maximale |
| tmin | Température minimale |
| soil | Humidité du sol |
| def | Déficit hydrique |
| pdsi | Indice de sécheresse Palmer |
| vpd | Déficit de pression de vapeur |
| ws | Vitesse du vent |

> **Note** : La variable 'ro' (runoff) n'est pas disponible dans le dataset TerraClimate sur Planetary Computer.

**Fichiers créés :**

| Fichier | Lignes | Colonnes |
|---------|--------|----------|
| terraclimate_features_training.csv | ~9300 | Latitude, Longitude, Sample Date + 10 variables |
| terraclimate_features_validation.csv | ~3700 | Latitude, Longitude, Sample Date + 10 variables |